In [37]:
%reload_ext autoreload
%autoreload 2

In [38]:
from catboost import CatBoostClassifier
import pandas as pd
import json
from pandas import json_normalize
from tqdm import tqdm_notebook as tqdm
import numpy as np
import ast
import time

import sys
import os
sys.path.append(os.path.join(sys.path[0], '../../core/'))

from train_utils import calculate_roi, get_winner_favorite, combine_df, parse_odds

In [39]:
fighters_df = pd.read_csv("../../data/0.fighters_raw.csv", index_col=0)
fighters_df["dateOfBirth"] = pd.to_datetime(fighters_df["dateOfBirth"])
fighters_cols = [
    "id",
    "name",
    "weight",
    "height",
    "armSpan",
    "legSwing",
    "weightCategory.id",
    "weightCategory.name",
    "dateOfBirth",
    "country",
    "city",
    "timezone",
]
fighters_df = fighters_df[fighters_cols]
fighters_df.set_index("id", inplace=True)
f_name_dict = fighters_df['name'].to_dict()

all_new_cols = json.load(open('../../data/Catboost_v1_0/all_new_cols_06.04.2021.txt', 'r'))
f_stats_events_cumulative = pd.read_csv('../../data/Catboost_v1_0/PROD_f_stats_events_cumulative_06.04.2021.csv', index_col=0)

In [41]:
f_stats_events_cumulative[f_stats_events_cumulative['fighterId'] == 3667]

,cumsum_duration,cumsum_winner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,...,hitsBody_PM,hitsLegs_PM,accentedHitsPositionDistance_PM,accentedHitsPositionClinch_PM,accentedHitsPositionParter_PM,knockdowns_PM,protectionPassage_PM,win_streak,loose_streak,fighterName
6813,240.0,0,11.0,7.0,0.0,0.0,0.0,0.0,8.0,4.0,...,0.0,0.25,0.75,0.25,0.0,0.0,0.0,0,1,Gaetano Pirrello


In [42]:
f_stats_events_cumulative[f_stats_events_cumulative['fighterName'] == 'Donald Cerrone']

,cumsum_duration,cumsum_winner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,...,hitsBody_PM,hitsLegs_PM,accentedHitsPositionDistance_PM,accentedHitsPositionClinch_PM,accentedHitsPositionParter_PM,knockdowns_PM,protectionPassage_PM,win_streak,loose_streak,fighterName
1645,300.0,1,69.0,30.0,3.0,2.0,0.0,0.0,68.0,29.0,...,0.600000,0.800000,5.600000,0.200000,0.000000,0.0,0.000000,1,0,Donald Cerrone
2376,463.0,2,74.0,33.0,3.0,2.0,3.0,0.0,72.0,31.0,...,0.388769,0.518359,3.758099,0.129590,0.129590,0.0,0.000000,2,0,Donald Cerrone
2418,691.0,3,122.0,64.0,4.0,3.0,4.0,0.0,106.0,48.0,...,0.694645,0.694645,3.473227,0.173661,0.520984,0.0,0.260492,3,0,Donald Cerrone
2533,991.0,4,188.0,104.0,5.0,3.0,4.0,0.0,172.0,88.0,...,0.908174,1.574168,4.722503,0.181635,0.423814,0.0,0.181635,4,0,Donald Cerrone
2592,1172.0,5,251.0,136.0,5.0,3.0,4.0,0.0,232.0,118.0,...,0.870307,1.638225,4.709898,0.204778,1.126280,0.0,0.153584,5,0,Donald Cerrone
2676,1314.0,6,274.0,145.0,5.0,3.0,5.0,0.0,255.0,127.0,...,0.867580,1.552511,4.520548,0.228311,1.050228,0.0,0.182648,6,0,Donald Cerrone
2751,1614.0,6,351.0,182.0,5.0,3.0,5.0,0.0,326.0,160.0,...,1.003717,1.412639,4.795539,0.297398,0.855019,0.0,0.148699,0,1,Donald Cerrone
2878,1914.0,7,419.0,213.0,6.0,3.0,5.0,0.0,394.0,191.0,...,1.253918,1.442006,4.984326,0.282132,0.721003,0.0,0.125392,1,0,Donald Cerrone
2989,1990.0,8,428.0,217.0,6.0,3.0,5.0,0.0,403.0,195.0,...,1.206030,1.417085,4.914573,0.271357,0.693467,0.0,0.120603,2,0,Donald Cerrone
3131,2145.0,8,440.0,221.0,6.0,3.0,5.0,0.0,415.0,199.0,...,1.146853,1.342657,4.671329,0.251748,0.643357,0.0,0.111888,0,1,Donald Cerrone


In [81]:
all_fightCols_list = all_new_cols['new_cumsum_colnames']+all_new_cols['new_accuracy_cols']+\
    all_new_cols['new_percent_cols']+all_new_cols['new_PM_cols']+all_new_cols['new_streak_cols'] + \
    ['count_of_fights', 'age', 'odds'] #,

# all_new_cols['new_difference_cols'] + 
# all_fightCols_list = [i + '_difference' for i in all_fightCols_list]
# all_fightCols_list += all_new_cols['f2_cumsum_col_names'] 
# all_fightCols_list += all_new_cols['f1_cumsum_col_names'] 
# all_fightCols_list += ['f1_age', 'f2_age']

In [82]:
import datetime

f1_name = 'Yoshihiro Akiyama'
f2_name = 'Jim Miller'
event_date = datetime.date(2021, 4, 5)


f1_birthDate = fighters_df[fighters_df['name'] == f1_name]['dateOfBirth']
f1_age = ((pd.to_datetime(event_date) - pd.to_datetime(f1_birthDate)) / 365).dt.days.values[0]

f2_birthDate = fighters_df[fighters_df['name'] == f2_name]['dateOfBirth']
f2_age = ((pd.to_datetime(event_date) - pd.to_datetime(f2_birthDate)) / 365).dt.days.values[0]


fighter1_stats = f_stats_events_cumulative[f_stats_events_cumulative['fighterName'] == f1_name]#.iloc[-1]
fighter1_stats_NaNs = (fighter1_stats[all_fightCols_list].isna().sum(axis=1) / fighter1_stats.shape[1])
fighter1_stats = pd.DataFrame(fighter1_stats[fighter1_stats_NaNs < 0.2].iloc[-1]).T.reset_index(drop=True)

fighter2_stats = f_stats_events_cumulative[f_stats_events_cumulative['fighterName'] == f2_name]#.iloc[-1]
fighter2_stats_NaNs = (fighter2_stats[all_fightCols_list].isna().sum(axis=1) / fighter2_stats.shape[1])
fighter2_stats = pd.DataFrame(fighter2_stats[fighter2_stats_NaNs < 0.2].iloc[-1]).T.reset_index(drop=True)
fighter2_stats

,cumsum_duration,cumsum_winner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,...,hitsBody_PM,hitsLegs_PM,accentedHitsPositionDistance_PM,accentedHitsPositionClinch_PM,accentedHitsPositionParter_PM,knockdowns_PM,protectionPassage_PM,win_streak,loose_streak,fighterName
0,7510.0,21,982.0,460.0,40.0,18.0,19.0,0.0,832.0,325.0,...,0.439414,0.60719,1.821571,0.431425,0.343542,0.0,0.151798,1,0,Jim Miller


In [83]:
X_df = pd.DataFrame(index=[0])

X_df = X_df.join(
    fighter1_stats[all_fightCols_list].add_prefix("f1_")) #, on="id"

X_df = X_df.join(
    fighter2_stats[all_fightCols_list].add_prefix("f2_"))

f1_odd, f2_odd = [1.1, 2.7]

X_df.loc[0, ['f1_age', 'f2_age', 'f1_odds', 'f2_odds']] = f1_age, f2_age, f1_odd, f2_odd
X_df[['weightCategory.name', 'city', 'country']] = ['Средний вес', 'Denver', 'USA']

X_df

,f1_cumsum_duration,f1_cumsum_winner,f1_cumsum_hitsTotal,f1_cumsum_hitsSuccessful,f1_cumsum_takedownTotal,f1_cumsum_takedownSuccessful,f1_cumsum_submissionAttempts,f1_cumsum_takeovers,f1_cumsum_accentedHitsTotal,f1_cumsum_accentedHitsSuccessful,...,f2_accentedHitsPositionDistance_PM,f2_accentedHitsPositionClinch_PM,f2_accentedHitsPositionParter_PM,f2_knockdowns_PM,f2_protectionPassage_PM,f2_win_streak,f2_loose_streak,f2_count_of_fights,f2_age,f2_odds
0,3458.0,11,585.0,292.0,15.0,12.0,8.0,0.0,454.0,190.0,...,1.821571,0.431425,0.343542,0.0,0.151798,1,0,33,37,2.7


In [90]:
def difference(events_df_joined, cols):
    df = events_df_joined.copy()
    # цикл заменяет столбцы характеристик каждого бойца столбцами разницы этих характеристик
    for col in cols:
        df[col+'_difference'] = events_df_joined['f1_'+col].astype(float) - events_df_joined['f2_'+col].astype(float)

    df['age_difference'] = events_df_joined['f1_age'] - events_df_joined['f2_age'] # не стал удалять столбцы с возрастом
    df['odds_difference'] = events_df_joined['f1_odds'] - events_df_joined['f2_odds'] # не стал удалять столбцы с возрастом
   
    return df

X_df_combined = difference(X_df, all_fightCols_list)
X_df_combined_reversed = X_df_combined.copy()
X_df_combined_reversed[all_fightCols_list] = X_df_combined_reversed[all_fightCols_list] * -1

KeyError: "None of [Index(['cumsum_duration', 'cumsum_winner', 'cumsum_hitsTotal',\n       'cumsum_hitsSuccessful', 'cumsum_takedownTotal',\n       'cumsum_takedownSuccessful', 'cumsum_submissionAttempts',\n       'cumsum_takeovers', 'cumsum_accentedHitsTotal',\n       'cumsum_accentedHitsSuccessful', 'cumsum_knockdowns',\n       'cumsum_protectionPassage', 'cumsum_hitsHeadTotal',\n       'cumsum_hitsHeadSuccessful', 'cumsum_hitsBodyTotal',\n       'cumsum_hitsBodySuccessful', 'cumsum_hitsLegsTotal',\n       'cumsum_hitsLegsSuccessful', 'cumsum_accentedHitsPositionDistanceTotal',\n       'cumsum_accentedHitsPositionDistanceSuccessful',\n       'cumsum_accentedHitsPositionClinchTotal',\n       'cumsum_accentedHitsPositionClinchSuccessful',\n       'cumsum_accentedHitsPositionParterTotal',\n       'cumsum_accentedHitsPositionParterSuccessful',\n       'cumsum_winMethods_[DEC]', 'cumsum_winMethods_[DQ]',\n       'cumsum_winMethods_[KO]', 'cumsum_winMethods_[SUB]', 'hits_accuracy',\n      

In [93]:
X_df_combined.columns.tolist()

['f1_cumsum_duration',
 'f1_cumsum_winner',
 'f1_cumsum_hitsTotal',
 'f1_cumsum_hitsSuccessful',
 'f1_cumsum_takedownTotal',
 'f1_cumsum_takedownSuccessful',
 'f1_cumsum_submissionAttempts',
 'f1_cumsum_takeovers',
 'f1_cumsum_accentedHitsTotal',
 'f1_cumsum_accentedHitsSuccessful',
 'f1_cumsum_knockdowns',
 'f1_cumsum_protectionPassage',
 'f1_cumsum_hitsHeadTotal',
 'f1_cumsum_hitsHeadSuccessful',
 'f1_cumsum_hitsBodyTotal',
 'f1_cumsum_hitsBodySuccessful',
 'f1_cumsum_hitsLegsTotal',
 'f1_cumsum_hitsLegsSuccessful',
 'f1_cumsum_accentedHitsPositionDistanceTotal',
 'f1_cumsum_accentedHitsPositionDistanceSuccessful',
 'f1_cumsum_accentedHitsPositionClinchTotal',
 'f1_cumsum_accentedHitsPositionClinchSuccessful',
 'f1_cumsum_accentedHitsPositionParterTotal',
 'f1_cumsum_accentedHitsPositionParterSuccessful',
 'f1_cumsum_winMethods_[DEC]',
 'f1_cumsum_winMethods_[DQ]',
 'f1_cumsum_winMethods_[KO]',
 'f1_cumsum_winMethods_[SUB]',
 'f1_hits_accuracy',
 'f1_takedown_accuracy',
 'f1_accented

In [86]:
clf = CatBoostClassifier()
clf.load_model('../../models/Catboost_v1_0/catboost_v1_0_06.04.2021.cat')
model_cols = clf.feature_names_
model_cols

['city',
 'country',
 'weightCategory.name',
 'f1_age',
 'f2_age',
 'f1_odds',
 'f2_odds',
 'cumsum_duration_difference',
 'cumsum_winner_difference',
 'cumsum_hitsTotal_difference',
 'cumsum_hitsSuccessful_difference',
 'cumsum_takedownTotal_difference',
 'cumsum_takedownSuccessful_difference',
 'cumsum_submissionAttempts_difference',
 'cumsum_takeovers_difference',
 'cumsum_accentedHitsTotal_difference',
 'cumsum_accentedHitsSuccessful_difference',
 'cumsum_knockdowns_difference',
 'cumsum_protectionPassage_difference',
 'cumsum_hitsHeadTotal_difference',
 'cumsum_hitsHeadSuccessful_difference',
 'cumsum_hitsBodyTotal_difference',
 'cumsum_hitsBodySuccessful_difference',
 'cumsum_hitsLegsTotal_difference',
 'cumsum_hitsLegsSuccessful_difference',
 'cumsum_accentedHitsPositionDistanceTotal_difference',
 'cumsum_accentedHitsPositionDistanceSuccessful_difference',
 'cumsum_accentedHitsPositionClinchTotal_difference',
 'cumsum_accentedHitsPositionClinchSuccessful_difference',
 'cumsum_ac

In [87]:
y_proba_straight = clf.predict_proba(X_df_combined[model_cols])[:,1]
y_proba_reversed = clf.predict_proba(X_df_combined_reversed[model_cols])[:,0]
y_proba = np.mean([y_proba_straight, y_proba_reversed])
y_proba

0.45979771144389986

In [88]:
X_df_combined[model_cols]

,city,country,weightCategory.name,f1_age,f2_age,f1_odds,f2_odds,cumsum_duration_difference,cumsum_winner_difference,cumsum_hitsTotal_difference,...,f1_cumsum_accentedHitsPositionDistanceTotal,f1_cumsum_accentedHitsPositionDistanceSuccessful,f1_cumsum_accentedHitsPositionClinchTotal,f1_cumsum_accentedHitsPositionClinchSuccessful,f1_cumsum_accentedHitsPositionParterTotal,f1_cumsum_accentedHitsPositionParterSuccessful,f1_cumsum_winMethods_[DEC],f1_cumsum_winMethods_[DQ],f1_cumsum_winMethods_[KO],f1_cumsum_winMethods_[SUB]
0,Denver,USA,Средний вес,45,37,1.1,2.7,-4052.0,-10.0,-397.0,...,316.0,102.0,19.0,12.0,119.0,76.0,5,0,5,7


In [89]:
X_df_combined_reversed[model_cols]

,city,country,weightCategory.name,f1_age,f2_age,f1_odds,f2_odds,cumsum_duration_difference,cumsum_winner_difference,cumsum_hitsTotal_difference,...,f1_cumsum_accentedHitsPositionDistanceTotal,f1_cumsum_accentedHitsPositionDistanceSuccessful,f1_cumsum_accentedHitsPositionClinchTotal,f1_cumsum_accentedHitsPositionClinchSuccessful,f1_cumsum_accentedHitsPositionParterTotal,f1_cumsum_accentedHitsPositionParterSuccessful,f1_cumsum_winMethods_[DEC],f1_cumsum_winMethods_[DQ],f1_cumsum_winMethods_[KO],f1_cumsum_winMethods_[SUB]
0,Denver,USA,Средний вес,45,37,1.1,2.7,4052.0,10.0,397.0,...,-316.0,-102.0,-19.0,-12.0,-119.0,-76.0,-5,0,-5,-7
